# Room for the river: Gorssel 

In [1]:
import sys

print("User Current Version:-", sys.version)

User Current Version:- 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [2]:
#imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from ema_workbench import (Model, CategoricalParameter,ScalarOutcome, IntegerParameter, RealParameter,
                           MultiprocessingEvaluator, ema_logging, Constant, Policy, Scenario,
                           perform_experiments, SequentialEvaluator,Constraint)
from problem_formulation import get_model_for_problem_formulation

from dike_model_function import DikeNetwork  # @UnresolvedImport

from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress,GenerationalBorg)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.analysis import plotting, plotting_util, parcoords, feature_scoring, prim
from ema_workbench import load_results 

from mpl_toolkits.mplot3d import Axes3D 

from sklearn import preprocessing

import time
import copy

import itertools

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import os
import functools

import random

In [3]:
from scipy.spatial.distance import pdist, squareform

def evaluate_diversity_single(indices, distances, weight=0.5, distance='euclidean'):
    '''
    takes the outcomes and selected scenario set (decision variables), 
    returns a single 'diversity' value for the scenario set.
    outcomes : outcomes dictionary of the scenario ensemble
    decision vars : indices of the scenario set
    weight : weight given to the mean in the diversity metric. If 0, only minimum; if 1, only mean
    '''
    i, j = [e for e in zip(*itertools.combinations(indices, 2))]
    subset_distances = distances[i, j]
    minimum = np.min(subset_distances)
    mean = np.mean(subset_distances)
    diversity = (1-weight)*minimum + weight*mean
    
    return [diversity]


def find_maxdiverse_scenarios(distances, combinations):
    scores = []
    for indices in combinations:
        diversity = evaluate_diversity_single(indices, distances)
        scores.append((diversity, indices))

    return scores

def optimize(scenario, nfe, model, converge_metrics, epsilons):


    with SequentialEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=epsilons,
                                     reference=scenario)
    return results, convergence


# Scenario Selection 

In [4]:
#running the model through EMA workbench
ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3) 


In [5]:
with SequentialEvaluator(dike_model) as evaluator:
    results_MORDM = evaluator.perform_experiments(scenarios=10, policies=5) 
    #s for number of scenarios, p for number of experiments 
    
experiments_MORDM, outcomes_MORDM = results_MORDM 
#s for number of scenarios, p for number of experiments 

[MainProcess/INFO] performing 10 scenarios * 5 policies * 1 model(s) = 50 experiments
100%|██████████████████████████████████████████| 50/50 [03:50<00:00,  4.61s/it]
[MainProcess/INFO] experiments finished


In [6]:
#save the initial results to be sure the original can be reused without having to run the model again 
raw_outcomes    = outcomes_MORDM.copy()
raw_experiments = experiments_MORDM.copy()

In [7]:
df_experiments = pd.DataFrame(experiments_MORDM)

df_outcomes =pd.DataFrame()
for outcome in outcomes_MORDM.keys():
    df_outcomes[outcome] = outcomes_MORDM[outcome]

df_experiments
df_experiments.columns

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model'],
      dtype='object')

In [8]:
experiments = df_experiments.copy()
outcomes = df_outcomes.copy()

In [9]:
outcomes.keys()

Index(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs'],
      dtype='object')

In [10]:
#determine thresholds 
max_evacuation_costs = 5000
max_expected_annual_damage = 1


In [11]:
outcomes[['A.4_Expected Annual Damage','A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
          'RfR Total Costs','Expected Evacuation Costs']]

outcomes_of_interest = outcomes[(outcomes['A.4_Expected Annual Damage']< max_expected_annual_damage) 
                                &( outcomes['Expected Evacuation Costs']<max_evacuation_costs)]

y = (outcomes['A.4_Expected Annual Damage']< max_expected_annual_damage)&( outcomes['Expected Evacuation Costs']<max_evacuation_costs)
#y

In [12]:
outcomes_of_interest.head()
#shape(outcomes_of_interest) 

,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Dike Investment Costs,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Dike Investment Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0.0,2.463876e+08,0.0,2.589972e+07,1.417416e+08,0.002801,0.0,1.478623e+08,0.0,0.0,2.842355e+07,0.0,0.0,1.352925e+08,0.0,883800000.0,1379.410586
1,0.0,2.463876e+08,0.0,2.943106e+06,1.417416e+08,0.000424,0.0,1.478623e+08,0.0,0.0,2.842355e+07,0.0,0.0,1.352925e+08,0.0,883800000.0,193.021557
3,0.0,2.463876e+08,0.0,4.287804e+07,1.417416e+08,0.004622,0.0,1.478623e+08,0.0,0.0,2.842355e+07,0.0,0.0,1.352925e+08,0.0,883800000.0,2286.996179
4,0.0,2.463876e+08,0.0,4.735309e+06,1.417416e+08,0.000415,0.0,1.478623e+08,0.0,0.0,2.842355e+07,0.0,0.0,1.352925e+08,0.0,883800000.0,191.405676
7,0.0,2.463876e+08,0.0,1.209086e+07,1.417416e+08,0.002050,0.0,1.478623e+08,0.0,0.0,2.842355e+07,0.0,0.0,1.352925e+08,0.0,883800000.0,978.249678


In [13]:
experiments.head()

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,113.0,245.389093,10,0.551521,155.740733,1.5,0.377944,276.429924,1.5,0.143022,...,4.0,2.0,3.0,4.0,3.0,9.0,3.0,5,0,dikesnet
1,64.0,286.591730,1.5,0.185900,164.493228,10,0.833925,296.658337,1.0,0.458840,...,4.0,2.0,3.0,4.0,3.0,9.0,3.0,6,0,dikesnet
2,1.0,340.626373,10,0.913145,204.119082,10,0.994911,115.315255,10,0.588699,...,4.0,2.0,3.0,4.0,3.0,9.0,3.0,7,0,dikesnet
3,27.0,184.236210,1.0,0.090132,273.135991,1.0,0.297745,142.490633,1.0,0.328798,...,4.0,2.0,3.0,4.0,3.0,9.0,3.0,8,0,dikesnet
4,130.0,72.576689,10,0.302170,292.225061,1.5,0.773962,46.438151,10,0.275168,...,4.0,2.0,3.0,4.0,3.0,9.0,3.0,9,0,dikesnet


In [14]:
experiments_of_interest = experiments.loc[y]
outcomes_df = pd.DataFrame({k:v[y] for k,v in outcomes.items()})

#normalize outcomes on unit interval to ensure equal weighting of outcomes
x = outcomes_df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_outcomes = pd.DataFrame(x_scaled, columns=outcomes_df.columns)

In [15]:
n_scen = experiments.loc[y].shape[0]
indices = range(n_scen)
set_size = 5

n_scen
combinations = itertools.combinations(indices, set_size)
combinations = list(combinations)

In [16]:
#combinations
print(len(combinations))

324632


In [17]:
sampled_combinations = random.sample(combinations, 100)

In [18]:
distances = squareform(pdist(normalized_outcomes.values))

cores = os.cpu_count()
partial_function = functools.partial(find_maxdiverse_scenarios, distances)


#Difference ProcessPool en ThreadPool 
# https://superfastpython.com/threadpoolexecutor-vs-processpoolexecutor/#Comparison_of_ThreadPoolExecutor_vs_ProcessPoolExecutor
#with ProcessPoolExecutor(max_workers=cores) as executor:
with ThreadPoolExecutor(max_workers=cores) as executor:
    worker_data = np.array_split(combinations, cores)
    results = [e for e in executor.map(partial_function, worker_data)]
    results = list(itertools.chain.from_iterable(results))

In [19]:
results.sort(key=lambda entry:entry[0], reverse=True)
most_diverse = results[0]
most_diverse

([1.840231314035805], array([ 2,  3,  7, 25, 32]))

In [20]:
selected = experiments.loc[most_diverse[1], ['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model']]
scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]

In [21]:
scenarios

[Scenario({'A.0_ID flood wave shape': 1.0, 'A.1_Bmax': 340.62637262693175, 'A.1_Brate': 10, 'A.1_pfail': 0.913145496620031, 'A.2_Bmax': 204.11908227240033, 'A.2_Brate': 10, 'A.2_pfail': 0.9949108542863907, 'A.3_Bmax': 115.315254772352, 'A.3_Brate': 10, 'A.3_pfail': 0.5886986842715578, 'A.4_Bmax': 99.43796664804417, 'A.4_Brate': 1.5, 'A.4_pfail': 0.13262790446644135, 'A.5_Bmax': 81.12625575700396, 'A.5_Brate': 1.5, 'A.5_pfail': 0.08075859704079529, 'discount rate 0': 1.5, 'discount rate 1': 4.5, 'discount rate 2': 3.5, '0_RfR 0': 1.0, '0_RfR 1': 0.0, '0_RfR 2': 0.0, '1_RfR 0': 0.0, '1_RfR 1': 1.0, '1_RfR 2': 1.0, '2_RfR 0': 0.0, '2_RfR 1': 0.0, '2_RfR 2': 0.0, '3_RfR 0': 1.0, '3_RfR 1': 1.0, '3_RfR 2': 1.0, '4_RfR 0': 0.0, '4_RfR 1': 0.0, '4_RfR 2': 0.0, 'A.1_DikeIncrease 0': 9.0, 'A.1_DikeIncrease 1': 8.0, 'A.1_DikeIncrease 2': 1.0, 'A.2_DikeIncrease 0': 0.0, 'A.2_DikeIncrease 1': 9.0, 'A.2_DikeIncrease 2': 1.0, 'A.3_DikeIncrease 0': 8.0, 'A.3_DikeIncrease 1': 10.0, 'A.3_DikeIncrease 2

# Mordm Run 

In [22]:
dike_model.outcomes = [ScalarOutcome('A.4_Expected Annual Damage', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.4_Dike Investment Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.4_Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('RfR Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('Expected Evacuation Costs', kind=ScalarOutcome.MINIMIZE)] 

In [28]:
#outcomes[['A.4_Expected Annual Damage','A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
 #         'RfR Total Costs','Expected Evacuation Costs']]

In [24]:
levers = ['A.0_ID flood wave shape', 
        'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
        'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 
        'A.3_Bmax', 'A.3_Brate', 'A.3_pfail', 
        'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 
        'A.5_Bmax', 'A.5_Brate', 'A.5_pfail', 
          'discount rate 0', 'discount rate 1','discount rate 2', 
          '0_RfR 0', '0_RfR 1', '0_RfR 2', 
          '1_RfR 0','1_RfR 1', '1_RfR 2', 
          '2_RfR 0', '2_RfR 1', '2_RfR 2', 
          '3_RfR 0','3_RfR 1', '3_RfR 2', 
          '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model']



levers_of_interest = ['A.4_Bmax', 'A.4_Brate', 'A.4_pfail',
        '0_RfR 0', '0_RfR 1', '0_RfR 2', 
        '1_RfR 0','1_RfR 1', '1_RfR 2', 
        '2_RfR 0','2_RfR 1', '2_RfR 2', 
        '3_RfR 0','3_RfR 1', '3_RfR 2', 
        '4_RfR 0','4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2']

names_outcomes = ['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs']

In [25]:
#Identify policies based on open exploration 

# policies = [Policy('policy 1', **{'A.0_ID flood wave shape':, 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
#        'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
#        'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
#        'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
#        'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
#        '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
#        '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
#        'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
#        'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
#        'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
#        'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
#        'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
#        'EWS_DaysToThreat', 'scenario', 'policy', 'model'})]


#            Policy('policy 2', **{'4_RfR 0':1,
#                                   '4_RfR 1':1,
#                                   '4_RfR 2':1,
#                                   'A.5_DikeIncrease 0':5}),
#            Policy('policy 3', **{'1_RfR 0':1,
#                                   '2_RfR 1':1,
#                                   '3_RfR 2':1,
#                                   'A.3_DikeIncrease 0':5})]



In [26]:
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [27]:
results = []
for scenario in scenarios:
    convergence_metrics = [HyperVolume(minimum=[0,0,0,0,0], maximum=[130000000,570000000,0.03,1900000000,45000]),
                           EpsilonProgress()]
    epsilons = [0.1,]*len(dike_model.outcomes)
    
    results.append(optimize(scenario, 1e4, dike_model, convergence_metrics, epsilons))

  0%|                                                | 0/10000 [00:00<?, ?it/s][MainProcess/ERROR] 'A.4_Expected Annual Damage'
Traceback (most recent call last):
  File "C:\Users\SIDTUDelft\miniconda3\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\SIDTUDelft\miniconda3\lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\SIDTUDelft\miniconda3\lib\site-packages\ema_workbench\em_framework\model.py", line 352, in run_model
    self.outcomes_output = outputs
  File "C:\Users\SIDTUDelft\miniconda3\lib\site-packages\ema_workbench\em_framework\model.py", line 67, in outcomes_output
    data = [outputs[var] for var in outcome.variable_name]
  File "C:\Users\SIDTUDelft\miniconda3\lib\site-packages\ema_workbench\em_framework\model.py", line 67, in <listcomp>
    data = [outputs[var] for var in outcome.variable_name]
KeyErro

EMAError: exception in run_model
Caused by: KeyError: 'A.4_Expected Annual Damage'

## Plots 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
for i, (_, convergence) in enumerate(results):
    ax1.plot(convergence.nfe, convergence.hypervolume, label=f'scenario {i}')
    ax2.plot(convergence.nfe, convergence.epsilon_progress, label=f'scenario {i}')

ax1.set_ylabel('hypervolume')
ax1.set_xlabel('nfe')
ax2.set_ylabel('$\epsilon$ progress')
ax2.set_xlabel('nfe')
fig.legend()
plt.show()

In [ ]:
colors = iter(sns.color_palette())

data = results[0][0].iloc[:, 5::]
limits = parcoords.get_limits(data)

limits.loc[0, ['inertia', 'reliability']] = 1
limits.loc[0, 'max_P'] = 4 # max over results based on quick inspection not shown here
limits.loc[0, 'utility'] = 1 # max over results based on quick inspection not shown here
limits.loc[1, :] = 0
paraxes = parcoords.ParallelAxes(limits)


for i, (result, _) in enumerate(results):
    color = next(colors)
    data = result.iloc[:, 5::]
    paraxes.plot(data, label=f'scenario {i}', color=color)

paraxes.legend()
plt.show()



## Deep Uncertainty 

In [ ]:
#specify the policies 

policies = []
for i, (result, _) in enumerate(results):
    result = result.iloc[:, 0:5]
    for j, row in result.iterrows():
        policy = Policy(f'scenario {i} option {j}', **row.to_dict())
        policies.append(policy)
        
## Our way: 

#Identify policies based on open exploration 

policies = [Policy('policy 1', **{'0_RfR 0':1,
                                  '0_RfR 1':1,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':5,
                                 'A.1_DikeIncrease 1':5}),
           Policy('policy 2', **{'4_RfR 0':1,
                                  '4_RfR 1':1,
                                  '4_RfR 2':1,
                                  'A.5_DikeIncrease 0':5}),
           Policy('policy 3', **{'1_RfR 0':1,
                                  '2_RfR 1':1,
                                  '3_RfR 2':1,
                                  'A.3_DikeIncrease 0':5})]
        

In [ ]:
with MultiprocessingEvaluator(model) as evaluator:
    reeevaluation_results = evaluator.perform_experiments(1000, policies=policies)

Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.



In [ ]:

experiments, outcomes = reeevaluation_results

#as defined earlier: 
thresholds = {'utility':0.75, 'inertia':0.99, 'reliability':0.8}

overall_scores = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    scores = {}
    for k, v in outcomes.items():
        try:
            n = np.sum(v[logical]>=thresholds[k])
        except KeyError:
            continue
        scores[k] = n/1000 
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T

In [ ]:
limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(overall_scores)
plt.show()

Alternative way?? 

In [ ]:
overall_scores = {}
regret = []
for scenario in experiments.scenario.unique():
    logical = experiments.scenario==scenario
    temp_results = {k:v[logical] for k,v in outcomes.items()}
    temp_results = pd.DataFrame(temp_results)
    temp_experiments = experiments[experiments.scenario==scenario]
        
    best = temp_results.max()
    best['max_P'] = temp_results.max_P.min()
    scenario_regret = a - best
    scenario_regret['policy'] = temp_experiments.policy.values    
    regret.append(scenario_regret)



In [ ]:
regret = pd.concat(regret)
maxregret = regret.groupby('policy').max()

In [ ]:
limits = parcoords.get_limits(maxregret)
paraxes = parcoords.ParallelAxes(maxregret)
paraxes.plot(maxregret)
plt.show()